In [1]:
pip install docxtpl

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for docxcompose: filename=docxcompose-1.4.0-py3-none-any.whl size=23267 sha256=7303976e9dd545f3d32a10b4a685050b02fa7b4b5e22a5adae2404ceaddeeb13
  Stored in directory: /Users/pxy/Library/Caches/pip/wheels/cd/f2/d5/ff7f0e5fad318c99a6c4668eda61ec0cdcb33ec6b5de5939a1
Successfully built docxcompose
Note: you may need to restart the kernel to use updated packages.


In [65]:
"""
先导入和检验一下模版
"""

from pathlib import Path
from docxtpl import DocxTemplate

# Jupyter 下直接用当前工作目录
BASE_DIR = Path.cwd()

# 模板路径
TEMPLATE_PATH = BASE_DIR / "template.docx"

# 尝试读取模板
try:
    template = DocxTemplate(TEMPLATE_PATH)
    print(f"[OK] 模板已成功读取: {TEMPLATE_PATH}")
except Exception as e:
    print(f"[ERROR] 模板读取失败: {e}")


[OK] 模板已成功读取: /Users/pxy/Desktop/HW_School_Application/template.docx


In [67]:
"""
换个随便的信息先试一下
"""



from pathlib import Path
from docxtpl import DocxTemplate

tpl_path = Path("/Users/pxy/Desktop/HW_School_Application/template.docx")

#  再做一次存在性确认并加载模板
print(f"[OK] 使用模板：{tpl_path}")
template = DocxTemplate(str(tpl_path))

# 上下文
context = {
    "university": "Renmin University of China",
    "program": "Master of Finance",
    "area": "finance",
    "journal1": "Review of Financial Studies",
    "journal2": "Journal of Finance",
    "journal3": "Management Science",
    "skills": "Python, SQL, Tableau",
    "career_goal": "quant researcher",
}

#渲染并保存
out_dir = Path.cwd() / "output"
out_dir.mkdir(exist_ok=True)
out_file = out_dir / "test_letter.docx"

template.render(context)
template.save(str(out_file))
print(f"[OK] 成功生成：{out_file}")

[OK] 使用模板：/Users/pxy/Desktop/HW_School_Application/template.docx
[OK] 成功生成：/Users/pxy/Desktop/HW_School_Application/output/test_letter.docx


In [69]:
"""
开始做大学清单csv
"""   



import pandas as pd
import random

# 设置随机种子以保证结果可复现 (可选)
# random.seed(42)

# -----------------------------------------------------
# 1. 提取并清理前90所大学名单（按新的提取规则）
# -----------------------------------------------------

# 提供的原始大学排名文本（按排名顺序）
raw_schools_data = """
1Department of Economics, Harvard University
2Economics Department, Massachusetts Institute of Technology (MIT)
3Department of Economics, University of California-Berkeley
4Department of Economics, University of Chicago
5Paris School of Economics
6Department of Economics, Princeton University
7Department of Economics, Stanford University
8Economics Department, Yale University
9Toulouse School of Economics (TSE)
10Department of Economics, Oxford University
11Department of Economics, School of Arts and Sciences, Columbia University
12Barcelona School of Economics (BSE)
13Department of Economics, New York University (NYU)
14Economics Department, Brown University
15Department of Economics, University of Pennsylvania
16Economics Department, London School of Economics (LSE)
17Department of Economics, Boston University
18Department of Economics, University College London (UCL)
19Department of Economics, University of California-San Diego (UCSD)
20Economics Department, University of Michigan
21Department of Economics, Northwestern University
22Economics Department, Dartmouth College
23Department of Economics, University of Southern California
24School of Economics and Management, Universiteit van Tilburg
25Vancouver School of Economics, University of British Columbia
26Department of Economics, University of California-Los Angeles (UCLA)
27Department of Economics, Duke University
28Economics Department, University of Wisconsin-Madison
29Institut für Volkswirtschaftslehre, Wirtschaftswissenschaftliche Fakutält, Universität Zürich
30Economics Department, Michigan State University
31Finance and Economics Department, Graduate School of Business, Columbia University
32Faculty of Economics, University of Cambridge
33Department of Economics, Boston College
34Department of Economics, University of Toronto
35Department of Economics, University of Warwick
36Department of Economics, Cornell University
37Economics Department, University of California-Davis
38Faculteit Economie en Bedrijfskunde, Rijksuniversiteit Groningen
39Department of Economics, Johns Hopkins University
40Dipartimento di Scienze Economiche, Alma Mater Studiorum - Università di Bologna
41Department of Economics, Sciences économiques, Sciences Po
42School of Economics, University of Nottingham
43Department of Economics, University of California-Irvine
44Department of Economics, Monash Business School, Monash University
45Faculteit Economie en Bedrijfswetenschappen, KU Leuven
46Institut for Økonomi, Aarhus Universitet
47Department of Economics, University of Maryland
48École d'Économie d'Aix-Marseille, Aix-Marseille Université
49School of Business and Economics, Vrije Universiteit Amsterdam
50Department of Economics, University of Notre Dame
51Department of Economics, Vanderbilt University
52Department of Economics, University of Texas-Austin
53Centro de Estudios Monetarios y Financieros (CEMFI)
54Department of Economics and Related Studies, University of York
55Økonomisk Institut, Københavns Universitet
56Faculteit der Economische Wetenschappen, Erasmus Universiteit Rotterdam
57College of Business and Economics, Australian National University
58Faculteit Economie en Bedrijfskunde, Universiteit van Amsterdam
59Department of Economics, University of Western Ontario
60Economics Department, Queen's University
61School of Economics, University of Queensland
62Department of Economics, University of Virginia
63Economics Department, Georgetown University
64Department of Economics, University of Minnesota
65Economics Department, Stern School of Business, New York University (NYU)
66Department of Economics, University of California-Santa Barbara (UCSB)
67Department of Economics, Washington University in St. Louis
68Department of Economics, Pennsylvania State University
69Department of Economics, University of Colorado
70Economics Department, University of California-Santa Cruz (UCSC)
71Department of Economics, Rutgers University-New Brunswick
72Volkswirtschaftliche Fakultät, Ludwig-Maximilians-Universität München
73Department of Economics, Ohio State University
74School of Economics, Universitat de Barcelona
75Department of Economics, Andrew Young School of Policy Studies, Georgia State University
76Dipartimenti e Istituti di Scienze Economiche, Università Cattolica del Sacro Cuore
77Fachbereich Wirtschaftswissenschaft, Goethe Universität Frankfurt am Main
78Eitan Berglas School of Economics, Tel Aviv University
79Departamento de Economía, Universidad Carlos III de Madrid
80Økonomisk institutt, Universitetet i Oslo
81School of Economics, UNSW Business School, UNSW Sydney
82Wirtschaftswissenschaftlicher Fachbereich, Rheinische Friedrich-Wilhelms-Universität Bonn
83Department of Economics, W.P. Carey School of Business, Arizona State University
84Department of Economics, Hebrew University of Jerusalem
85Facoltà di Economia, Università degli Studi di Roma "Tor Vergata"
86Department of Economics, George Washington University
87Department of Economics, Mitch Daniels School of Business, Purdue University
88School of Economics, Faculty of Arts and Social Sciences, University of Sydney
89Department of Economics, McGill University
90Department of Economics, University of Washington
"""

all_top90_schools = []
for line in raw_schools_data.strip().split('\n'):
    # 1. 移除行首排名数字和空格
    school_info = line.split(' ', 1)[1].strip()

    # 2. 实施提取规则：有逗号的选逗号后面的，没逗号的保留所有
    if ',' in school_info:
        # 选最后一个逗号后面的所有内容，并去除首尾空格
        final_name = school_info.split(',')[-1].strip()
    else:
        # 没逗号，保留所有内容
        final_name = school_info.strip()

    # 3. 清理：移除括号中的缩写（例如，将 "University of Chicago (UC)" -> "University of Chicago"）
    # 确保最终输出的是全名，例如 "MIT" 要变成 "Massachusetts Institute of Technology"
    if '(' in final_name and ')' in final_name:
        # 如果逗号后的部分仍然包含括号，则移除括号及内容
        start_index = final_name.find('(')
        final_name = final_name[:start_index].strip()
    
    # 针对几个常见的缩写进行回溯替换，确保全名
    if final_name == "MIT": final_name = "Massachusetts Institute of Technology"
    elif final_name == "LSE": final_name = "London School of Economics"
    elif final_name == "NYU": final_name = "New York University"
    
    all_top90_schools.append(final_name.strip())


# 确保名单是90所大学
all_top90_schools = [s for s in all_top90_schools if s][:90] 


# -----------------------------------------------------
# 2. 划分区间 (Rank 1-30, 31-60, 61-90)
# -----------------------------------------------------
schools_rank1_30 = all_top90_schools[:30]
schools_rank31_60 = all_top90_schools[30:60]
schools_rank61_90 = all_top90_schools[60:90]

# -----------------------------------------------------
# 3. 从每个区间随机抽取10所大学
# -----------------------------------------------------
# 使用 random.sample 进行随机抽取
schools_top30_random = random.sample(schools_rank1_30, 10)
schools_30_60_random = random.sample(schools_rank31_60, 10)
schools_60_90_random = random.sample(schools_rank61_90, 10)

# -----------------------------------------------------
# 4. 组合成 DataFrame
# -----------------------------------------------------
final_schools_list = schools_top30_random + schools_30_60_random + schools_60_90_random

df_schools = pd.DataFrame({
    "university": final_schools_list,
    # 保持原有的分组名称
    "group": (["Top30"] * len(schools_top30_random)) +
             (["Top30-60"] * len(schools_30_60_random)) +
             (["Top60-90"] * len(schools_60_90_random))
})

# -----------------------------------------------------
# 5. 保存 CSV (路径和文件名不变)
# -----------------------------------------------------
df_schools.to_csv("schools_all.csv", index=False)

print("已成功将随机抽取的30所大学保存为 schools_all.csv")
print("\n--- 随机抽取的30所大学清单---")
print(df_schools)

# -----------------------------------------------------
# 6. 验证读取
# -----------------------------------------------------
df_univ = pd.read_csv("schools_all.csv")

已成功将随机抽取的30所大学保存为 schools_all.csv

--- 随机抽取的30所大学清单---
                                university     group
0                        Oxford University     Top30
1               University of Pennsylvania     Top30
2                     Princeton University     Top30
3                      Columbia University     Top30
4                  Northwestern University     Top30
5                       Harvard University     Top30
6                University College London     Top30
7     University of California-Los Angeles     Top30
8                      Stanford University     Top30
9                        Boston University     Top30
10                               KU Leuven  Top30-60
11                University of Nottingham  Top30-60
12                  Københavns Universitet  Top30-60
13                   University of Warwick  Top30-60
14                       Monash University  Top30-60
15            Vrije Universiteit Amsterdam  Top30-60
16                             Sciences Po  

In [71]:
import pandas as pd

# 1. program -> area & career_goal
program_mapping = pd.DataFrame({
    "program": [
        "PhD in Economics program",
        "Master of Finance program",
        "Master of Information Management program"
    ],
    "area": [
        "Economics",
        "Finance",
        "Information Management"
    ],
    "career_goal": [
        "Professor",
        "Quant Researcher",
        "Data Analyst"
    ]
})
program_mapping.to_csv("program_mapping.csv", index=False)

In [73]:
# 2. area -> journals
area_journals = pd.DataFrame({
    "area": ["Economics", "Economics", "Economics",
             "Finance", "Finance", "Finance",
             "Information Management", "Information Management", "Information Management"],
    "journal": [
        "American Economic Review (AER)", "Quarterly Journal of Economics (QJE)", "Journal of Political Economy (JPE)",
        "Journal of Finance (JF)", "Review of Financial Studies (RFS)", "Journal of Financial Economics (JFE)",
        "MIS Quarterly (MISQ)", "Information Systems Research (ISR)", "Journal of Management Information Systems (JMIS)"
    ]
})
area_journals.to_csv("area_journals.csv", index=False)

In [75]:
# 3. career_goal -> skills
career_skills = pd.DataFrame({
    "career_goal": [
        "Professor", "Professor", "Professor", "Professor",
        "Quant Researcher", "Quant Researcher", "Quant Researcher", "Quant Researcher",
        "Data Analyst", "Data Analyst", "Data Analyst", "Data Analyst"
    ],
    "skill": [
        "Academic Writing", "Econometrics", "Data Analysis", "Stata",
        "Python", "SQL", "Machine Learning", "Mathematics",
        "Excel", "PowerBI", "Data Visualization", "Business Analysis"
    ]
})
career_skills.to_csv("career_skills.csv", index=False)

print("[OK] 已生成 program_mapping.csv, area_journals.csv, career_skills.csv")

[OK] 已生成 program_mapping.csv, area_journals.csv, career_skills.csv


In [77]:
import pandas as pd

# 读取 CSV 文件
df_univ = pd.read_csv("schools_all.csv")
df_program = pd.read_csv("program_mapping.csv")
df_area = pd.read_csv("area_journals.csv")
df_career = pd.read_csv("career_skills.csv")

print("=== Universities ===")
print(df_univ.head(10))   # 打印前10所，避免太长

print("\n=== Program Mapping ===")
print(df_program)

print("\n=== Area -> Journals Mapping ===")
print(df_area)

print("\n=== Career Goal -> Skills Mapping ===")
print(df_career)


=== Universities ===
                             university  group
0                     Oxford University  Top30
1            University of Pennsylvania  Top30
2                  Princeton University  Top30
3                   Columbia University  Top30
4               Northwestern University  Top30
5                    Harvard University  Top30
6             University College London  Top30
7  University of California-Los Angeles  Top30
8                   Stanford University  Top30
9                     Boston University  Top30

=== Program Mapping ===
                                    program                    area  \
0                  PhD in Economics program               Economics   
1                 Master of Finance program                 Finance   
2  Master of Information Management program  Information Management   

        career_goal  
0         Professor  
1  Quant Researcher  
2      Data Analyst  

=== Area -> Journals Mapping ===
                     area     

In [79]:

"""
做一个替换demo试一下csv读不读得对

"""



from pathlib import Path
import pandas as pd
from docxtpl import DocxTemplate

# === 路径 & 模板 ===
BASE = Path.cwd()
TEMPLATE_PATH = BASE / "template.docx"
assert TEMPLATE_PATH.exists(), "template.docx 不在当前目录"
template = DocxTemplate(str(TEMPLATE_PATH))

# === 读取 CSV（你之前已经生成）===
df_univ   = pd.read_csv("schools_all.csv")        # 至少有一列 university
df_prog   = pd.read_csv("program_mapping.csv")     # program, area, career_goal
df_area   = pd.read_csv("area_journals.csv")       # area, journal（长表）
df_career = pd.read_csv("career_skills.csv")       # career_goal, skill（长表）

# --- 1) 选一个学校 & 一个 program（你可改成别的）---
university = df_univ["university"].iloc[0]  # 取第一所学校
program    = df_prog["program"].iloc[0]     # 取第一个项目（PhD in Economics program）

# --- 2) program -> area / career_goal ---
row_prog = df_prog[df_prog["program"] == program].iloc[0]
area = row_prog["area"]
career_goal = row_prog["career_goal"]

# --- 3) area -> journals（从长表取3本期刊）---
journals = df_area[df_area["area"].str.lower() == str(area).lower()]["journal"].tolist()
# 若该 area 少于3条，用兜底补齐
while len(journals) < 3:
    journals.append(f"Top Journal {len(journals)+1}")
journal1, journal2, journal3 = journals[:3]

# --- 4) career_goal -> skills（从长表收集并拼成字符串）---
skills_list = df_career[df_career["career_goal"].str.lower() == str(career_goal).lower()]["skill"].tolist()
skills_str = ", ".join(skills_list) if skills_list else "Python, SQL"

# --- 5) 组织上下文（与模板 {{ }} 一致）---
context = {
    "university": university,
    "program": program,
    "area": area,
    "journal1": journal1,
    "journal2": journal2,
    "journal3": journal3,
    "career_goal": career_goal,
    "skills": skills_str,
}

# --- 6) 渲染并保存 ---
out_dir = BASE / "output"
out_dir.mkdir(exist_ok=True)
out_file = out_dir / "demo_single.docx"

doc = DocxTemplate(str(TEMPLATE_PATH))
doc.render(context)
doc.save(str(out_file))

print("[OK] 已生成：", out_file)
print("学校：", university)
print("项目：", program)
print("方向：", area, "→", [journal1, journal2, journal3])
print("目标：", career_goal, "→", skills_str)


[OK] 已生成： /Users/pxy/Desktop/HW_School_Application/output/demo_single.docx
学校： Oxford University
项目： PhD in Economics program
方向： Economics → ['American Economic Review (AER)', 'Quarterly Journal of Economics (QJE)', 'Journal of Political Economy (JPE)']
目标： Professor → Academic Writing, Econometrics, Data Analysis, Stata


In [81]:
from pathlib import Path
import pandas as pd
from docxtpl import DocxTemplate

# === 准备路径和模板 ===
BASE = Path.cwd()
TEMPLATE_PATH = BASE / "template.docx"
template = DocxTemplate(str(TEMPLATE_PATH))

# === 读取数据 ===
df_univ = pd.read_csv("schools_all.csv")
df_prog   = pd.read_csv("program_mapping.csv")
df_area   = pd.read_csv("area_journals.csv")
df_career = pd.read_csv("career_skills.csv")

def generate_one(school_idx=0, program_idx=0):
    """
    根据学校序号和项目序号生成一封申请信
    school_idx: 学校编号 (0 开始)
    program_idx: 项目编号 (0 开始)
    """
    # --- 学校 ---
    university = df_univ["university"].iloc[school_idx]

    # --- 项目 -> area, career_goal ---
    program_row = df_prog.iloc[program_idx]
    program = program_row["program"]
    area = program_row["area"]
    career_goal = program_row["career_goal"]

    # --- journals ---
    journals = df_area[df_area["area"].str.lower() == area.lower()]["journal"].tolist()
    while len(journals) < 3:
        journals.append(f"Top Journal {len(journals)+1}")
    journal1, journal2, journal3 = journals[:3]

    # --- skills ---
    skills_list = df_career[df_career["career_goal"].str.lower() == career_goal.lower()]["skill"].tolist()
    skills_str = ", ".join(skills_list) if skills_list else "Python, SQL"

    # --- context ---
    context = {
        "name": "Cici Pan",
        "university": university,
        "program": program,
        "area": area,
        "journal1": journal1,
        "journal2": journal2,
        "journal3": journal3,
        "career_goal": career_goal,
        "skills": skills_str,
    }

    # --- 渲染并保存 ---
    out_dir = BASE / "output"
    out_dir.mkdir(exist_ok=True)
    fname = f"{university.replace(' ','_')}__{program.replace(' ','_')}.docx"
    out_file = out_dir / fname

    doc = DocxTemplate(str(TEMPLATE_PATH))
    doc.render(context)
    doc.save(str(out_file))

    print(f"[OK] 已生成：{out_file}")
    print("学校：", university)
    print("项目：", program)
    print("方向：", area, "→", [journal1, journal2, journal3])
    print("目标：", career_goal, "→", skills_str)

    return out_file


In [83]:
generate_one(23, 2)   # 这里就是 "Universidad Carlos III de Madrid" + 第3个项目



[OK] 已生成：/Users/pxy/Desktop/HW_School_Application/output/Goethe_Universität_Frankfurt_am_Main__Master_of_Information_Management_program.docx
学校： Goethe Universität Frankfurt am Main
项目： Master of Information Management program
方向： Information Management → ['MIS Quarterly (MISQ)', 'Information Systems Research (ISR)', 'Journal of Management Information Systems (JMIS)']
目标： Data Analyst → Excel, PowerBI, Data Visualization, Business Analysis


PosixPath('/Users/pxy/Desktop/HW_School_Application/output/Goethe_Universität_Frankfurt_am_Main__Master_of_Information_Management_program.docx')

In [85]:
def generate_one(school_idx: int, program_idx: int, seq: int | None = None):
    """
    生成一封申请信。
    - school_idx: 学校序号（0-based）
    - program_idx: 项目序号（0-based）
    - seq: 可选的显示序号（1-based），若提供会用于文件名前缀，如 [005]
    """
    # --- 边界校验 ---
    n_schools = len(df_univ); n_programs = len(df_prog)
    if not (0 <= school_idx < n_schools):
        raise IndexError(f"school_idx 越界：{school_idx}，范围 0..{n_schools-1}")
    if not (0 <= program_idx < n_programs):
        raise IndexError(f"program_idx 越界：{program_idx}，范围 0..{n_programs-1}")

    # --- 取值 ---
    university = df_univ["university"].iloc[school_idx]
    program_row = df_prog.iloc[program_idx]
    program = program_row["program"]; area = program_row["area"]; career_goal = program_row["career_goal"]

    # --- 映射期刊 ---
    journals = df_area[df_area["area"].str.lower() == str(area).lower()]["journal"].tolist()
    while len(journals) < 3: journals.append(f"Top Journal {len(journals)+1}")
    j1, j2, j3 = journals[:3]

    # --- 映射技能 ---
    skills_list = df_career[df_career["career_goal"].str.lower() == str(career_goal).lower()]["skill"].tolist()
    skills_str = ", ".join(skills_list) if skills_list else "Python, SQL"

    # --- 渲染 ---
    out_dir = BASE / "output"; out_dir.mkdir(exist_ok=True)
    safe = lambda s: "".join(ch for ch in str(s) if ch.isalnum() or ch in " _-").strip().replace(" ", "_")

    prefix = f"[{seq:03d}]__" if isinstance(seq, int) and seq >= 1 else ""
    out_file = out_dir / f"{prefix}{safe(university)}__{safe(program)}.docx"

    doc = DocxTemplate(str(TEMPLATE_PATH))
    doc.render({
        "university": university,
        "program": program,
        "area": area,
        "journal1": j1, "journal2": j2, "journal3": j3,
        "career_goal": career_goal,
        "skills": skills_str,
    })
    doc.save(str(out_file))

    print(f"[OK] 已生成：{out_file}")
    return out_file


In [127]:
out_count = 0
seq = 1  # 1-based 序号，文件名前缀会显示为 [001]、[002] …

for i in range(len(df_univ)):        # 遍历 30 所学校
    for j in range(len(df_prog)):    # 遍历 3 个项目
        generate_one(i, j, seq=seq)  # 调用之前的函数，传入序号
        seq += 1
        out_count += 1

print(f"[OK] 共生成 {out_count} 封")

[OK] 已生成：/Users/pxy/Desktop/Python/lecture-python-programming.notebooks-main/python homework 1/output/[001]__Harvard_University__PhD_in_Economics_program.docx
[OK] 已生成：/Users/pxy/Desktop/Python/lecture-python-programming.notebooks-main/python homework 1/output/[002]__Harvard_University__Master_of_Finance_program.docx
[OK] 已生成：/Users/pxy/Desktop/Python/lecture-python-programming.notebooks-main/python homework 1/output/[003]__Harvard_University__Master_of_Information_Management_program.docx
[OK] 已生成：/Users/pxy/Desktop/Python/lecture-python-programming.notebooks-main/python homework 1/output/[004]__Yale_University__PhD_in_Economics_program.docx
[OK] 已生成：/Users/pxy/Desktop/Python/lecture-python-programming.notebooks-main/python homework 1/output/[005]__Yale_University__Master_of_Finance_program.docx
[OK] 已生成：/Users/pxy/Desktop/Python/lecture-python-programming.notebooks-main/python homework 1/output/[006]__Yale_University__Master_of_Information_Management_program.docx
[OK] 已生成：/Users/pxy/D

In [129]:
#可以在这个索引查看学校和专业的序号 方便查找

def list_options():
    print("Schools:")
    for i, u in enumerate(df_univ["university"].tolist()):
        print(f"[{i:02d}] {u}")
    print("\nPrograms:")
    for j, p in enumerate(df_prog["program"].tolist()):
        print(f"[{j}] {p}")

list_options()


Schools:
[00] Harvard University
[01] Yale University
[02] New York University
[03] London School of Economics
[04] Oxford University
[05] Princeton University
[06] Stanford University
[07] MIT
[08] University of California, Berkeley
[09] Paris School of Economics
[10] University of Cambridge
[11] University of Toronto
[12] Cornell University
[13] Johns Hopkins University
[14] University of Warwick
[15] Sciences Po
[16] University of Amsterdam
[17] University of Maryland
[18] University of Texas-Austin
[19] Erasmus University Rotterdam
[20] Ludwig-Maximilians-Universität München
[21] Goethe Universität Frankfurt
[22] Universitat de Barcelona
[23] Universidad Carlos III de Madrid
[24] University of Oslo
[25] University of Bonn
[26] Hebrew University of Jerusalem
[27] University of Sydney
[28] McGill University
[29] University of Washington

Programs:
[0] PhD in Economics program
[1] Master of Finance program
[2] Master of Information Management program
